In [1]:
# # How to get the data
# !wget http://files.fast.ai/data/imagenet-sample-train.tar.gz
# !gunzip imagenet-sample-train.tar.gz
# !tar xvf imagenet-sample-train.tar

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import foolbox
import tqdm
import imageio
from torchvision import transforms
from torch.autograd import Variable

In [3]:
# load dataset
imagenet_data = torchvision.datasets.ImageFolder('/data/imagenet_sample_train/', transform=transforms.ToTensor(), )

# load model
model = torchvision.models.resnet50(pretrained=True, )
model.eval()
model.to("cpu")

# prep Foolbox model
mean = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
std = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))
fmodel = foolbox.models.PyTorchModel(model, bounds=(0,1), num_classes=1000, preprocessing=(mean, std), device='cuda')

# define attack model
attack = foolbox.attacks.FGSM(fmodel)

In [4]:
# image, label = imagenet_data[0]; 
# print('predicted class', np.argmax(fmodel.forward(np.expand_dims(image, 0))))
# plt.figure()
# plt.imshow(image[0,:,:])

# adversarial = attack(image.numpy(), label); 
# print('adversarial class', np.argmax(fmodel.forward(np.expand_dims(adversarial, 0))))
# plt.figure()
# plt.imshow(adversarial[0,:,:])

In [5]:
# Accuracy counter
correct = 0
classifier_failed = 0
adversary_succeeded = 0
adversary_failed = 0
adv_examples = []
epsilon = 0.5

attack = foolbox.attacks.FGSM(fmodel)
attack_name = "fgsm"
attack_image_folder = "./imagenet/" + attack_name + "/"
!mkdir -p $attack_image_folder

# with open("./checkpoint.txt") as file:
#     last_checked = int(file.read())

# Loop over all examples in test set
for i in tqdm.tqdm_notebook(range(len(imagenet_data))):

#     with open("./checkpoint.txt","w") as file:
#         file.write(str(i))
    
    # Send the data and label to the device
    image, target = imagenet_data[i]

    if image.shape[1] > 1000 or image.shape[2] > 1000:
        del image
#         i += 1
        continue
    
    # Forward pass the data through the model
    output = fmodel.forward(np.expand_dims(image, 0, ))
    init_pred = np.argmax(output) # get the index of the max log-probability

    # If the initial prediction is wrong, dont bother attacking, just move on
    if init_pred != target:
        classifier_failed += 1
        continue

    # Call FGSM Attack
    adversarial = attack(image.numpy(), target)
    del image
    if adversarial is None:
        adversary_failed += 1
        del adversarial
        continue
    
    # Re-classify the perturbed image
    adv_output = fmodel.forward(np.expand_dims(adversarial, 0))
    perturbed_pred = np.argmax(adv_output)

    # Check for success
    if perturbed_pred == target:
        correct += 1
    else:
        adversary_succeeded += 1
        adversarial = np.rollaxis(adversarial, 0, 3)
        adversarial = adversarial * 255
        adversarial = adversarial.astype(np.uint8)
        imageio.imsave(attack_image_folder + attack_name + "_" + str(i) + ".png", adversarial)
        del adversarial
        with open(attack_image_folder + attack_name + "_" + str(i) + ".txt", "w") as file:
            file.write(str(perturbed_pred))
    
            
# Calculate final accuracy for this epsilon
final_acc = correct/float(len(imagenet_data))
print("Test Accuracy = {} / {} = {}".format(correct, len(imagenet_data), final_acc))


Test Accuracy = 0 / 19439 = 0.0


In [8]:
from pathlib import Path
len(list(Path('./imagenet/fgsm/').glob("*"))) / 2

59.0